_________

**Классификация токсичных комментариев с использованием BERT**

_________

**Описание проекта.**

Интернет-магазин «Викишоп» запускает новый сервис, позволяющий пользователям редактировать и дополнять описания товаров в стиле вики-сообществ. Клиенты могут предлагать правки и комментировать изменения других пользователей. Для обеспечения качества контента требуется инструмент, который автоматически выявляет токсичные комментарии и отправляет их на модерацию. Цель проекта — разработать модель машинного обучения для классификации комментариев на позитивные и негативные с метрикой качества F1 не менее 0.75. В проекте используется предобученная модель BERT для создания эмбеддингов текстов, а также различные алгоритмы машинного обучения для классификации.

__________

**Задачи проекта.**

1. Загрузить и изучить данные из файла `toxic_comments.csv`.
2. Выполнить предобработку текстовых данных, включая токенизацию и создание эмбеддингов с использованием модели BERT.
3. Подготовить данные для обучения моделей.
5. Обучить и сравнить несколько моделей машинного обучения.
6. Выбрать лучшую модель на основе метрики F1 и протестировать её на тестовой выборке.
7. Сформулировать итоговые выводы.

____________

**Исходные данные.**

Данные находятся в файле `/datasets/toxic_comments.csv`.  
- Столбец `text`: текст комментария.  
- Столбец `toxic`: целевой признак (0 — позитивный, 1 — токсичный).  

____________

**Данное исследование разделим на несколько частей.**

- [**Шаг 1. Загрузка и изучение данных.**](#section1)

- [**Шаг 2. Создание эмбедингов.**](#section2)
  
- [**Шаг 3. Обучение моделей.**](#section3)

- [**Шаг 4. Итоговый вывод.**](#section4)

---------

In [11]:
import os
import numpy as np
import pandas as pd

import torch
import transformers
from tqdm import notebook

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.svm import SVC

import optuna
from lightgbm import LGBMClassifier


# Загрузка данных.

In [3]:
df_pth1 = '/Users/ruslanminacov/Downloads/toxic_comments.csv'

if os.path.exists(df_pth1):
    df_tweets = pd.read_csv(df_pth1)
else:
    print('Something is wrong with toxic_comments.csv')




if 'df_tweets' in locals():
    print(f"df_tweets loaded with shape: {df_tweets.shape}")

df_tweets loaded with shape: (159292, 3)


In [22]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [23]:
df_tweets.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


**Промежуточный вывод:**

1. **Успешная загрузка данных**:
   - Датасет загружен с использованием библиотеки `pandas` из файла.
   - Проверка существования файла выполнена успешно, ошибок при загрузке не возникло.

2. **Характеристики датасета**:
   - Датасет `df_tweets` содержит **159292 строки** и **3 столбца**:
     - `Unnamed: 0`: индекс (тип данных — `int64`).
     - `text`: текст комментариев (тип данных — `object`, строка).
     - `toxic`: метка токсичности (тип данных — `int64`, бинарная переменная).
   - Все столбцы не содержат пропущенных значений (`Non-Null Count: 159292` для каждого столбца).
   - Объем памяти, занимаемый датасетом, составляет около **3.6+ МБ**.

3. **Предварительный просмотр данных**:
   - Первые пять строк датасета показывают, что столбец `text` содержит текстовые комментарии, а столбец `toxic` имеет значение `0` (нетоксичные комментарии) для этих примеров.
   - Столбец `Unnamed: 0` является техническим индексом и, вероятно, не несет полезной информации для анализа.



# Создание эмбедингов.

Для токенизации и создании эмбедингов выберем модель 'unitary/toxic-bert', данная модель уже предобучена для определения токсичных комментариев. Процесс создания эмбедингов запустим на GPU для меньшей затраты времени.

In [3]:
tokenizer = transformers.BertTokenizer.from_pretrained('unitary/toxic-bert')
model = transformers.BertModel.from_pretrained('unitary/toxic-bert')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [4]:
tokenized = df_tweets['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True))


max_len = max(len(i) for i in tokenized.values)


padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
attention_mask = np.where(padded != 0, 1, 0)

In [5]:
batch_size = 100
embeddings = []
model.eval()
for i in notebook.tqdm(range(0, padded.shape[0], batch_size)):
    batch = torch.LongTensor(padded[i:i+batch_size]).to(device)
    attention_mask_batch = torch.LongTensor(attention_mask[i:i+batch_size]).to(device)
    
    with torch.no_grad():
        batch_embeddings = model(batch, attention_mask=attention_mask_batch)
    
    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())


features = np.concatenate(embeddings)


  0%|          | 0/1593 [00:00<?, ?it/s]

features_df = pd.DataFrame(features)
features_df.to_csv('toxic_features_en.csv', index=False)

features_df = pd.read_csv('toxic_features_en.csv')

Для обучения моделий дополнительно созданим валидационную выборку(вместо использования метода cross_val_score) в целях экономии ресурсов. Разделим данные на выборки:

In [6]:
X_train_val, X_test, y_train_val, y_test = train_test_split(features_df, df_tweets['toxic'], test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

**Промежуточный вывод**

1. Создание эмбеддингов
- **Выбор модели**: Использована предобученная модель `unitary/toxic-bert`, оптимизированная для определения токсичных комментариев.
- **Токенизация**:
  - Применен `BertTokenizer` с максимальной длиной последовательности 512 токенов.
  - Учтены специальные токены и применено усечение (truncation) для длинных текстов.
- **Ускорение вычислений**:
  - Процесс выполнен на GPU (при наличии) для оптимизации времени.
  - Модель переведена в режим оценки (`model.eval()`) для исключения вычисления градиентов.
- **Обработка данных**:
  - Токенизированные последовательности дополнены (padding) до максимальной длины.
  - Созданы маски внимания (attention masks) для учета реальных токенов.
  - Эмбеддинги извлечены батчами (размер батча — 100).
- **Результат**:
  - Полученные эмбеддинги объединены в массив `features`.

2. Подготовка данных для обучения
- **Разделение выборок**:
  - Данные разделены на тренировочную+валидационную (`X_train_val`, 80%) и тестовую (`X_test`, 20%) с `random_state=42` для воспроизводимости.
  - Из `X_train_val` выделены тренировочная (`X_train`, 80%) и валидационная (`X_val`, 20%) выборки.
  - Целевая переменная `toxic` разделена на `y_train`, `y_val` и `y_test` соответственно.
- **Цель**:
  - Экономия ресурсов за счет использования валидационной выборки вместо кросс-валидации.
  - Обеспечение воспроизводимости и структурированности данных для последующего обучения модели.

# Обучение моделей.

## Модель LogisticRegression.

In [8]:
def objective(trial):
    
    C = trial.suggest_float("C", 1e-5, 1e2, log=True)
    solver = trial.suggest_categorical("solver", ["lbfgs", "liblinear", "saga"])
    max_iter = trial.suggest_int("max_iter", 100, 1000)
    penalty = trial.suggest_categorical("penalty", ["l1", "l2"]) if solver in ["liblinear", "saga"] else "l2"

    
    model = LogisticRegression(
        C=C,
        solver=solver,
        penalty=penalty,
        max_iter=max_iter,
        random_state=42
    )
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    f1_val = f1_score(y_val, y_pred)

    return f1_val

In [9]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)  

# Вывод лучших гиперпараметров
print("Лучшие гиперпараметры: ", study.best_params)
print("Лучшая точность: ", study.best_value)

[I 2025-07-10 20:16:30,540] A new study created in memory with name: no-name-dbbf58ba-9ddd-4163-afd2-9931d1302b57
[I 2025-07-10 20:16:51,997] Trial 0 finished with value: 0.9502912621359223 and parameters: {'C': 0.05949001406067821, 'solver': 'liblinear', 'max_iter': 857, 'penalty': 'l2'}. Best is trial 0 with value: 0.9502912621359223.
[I 2025-07-10 20:16:53,866] Trial 1 finished with value: 0.9414566382218694 and parameters: {'C': 2.5636704145768908e-05, 'solver': 'lbfgs', 'max_iter': 870}. Best is trial 0 with value: 0.9502912621359223.
[I 2025-07-10 20:17:02,872] Trial 2 finished with value: 0.9451279047061121 and parameters: {'C': 0.00010818672273001525, 'solver': 'liblinear', 'max_iter': 692, 'penalty': 'l2'}. Best is trial 0 with value: 0.9502912621359223.
/opt/anaconda3/envs/practicum/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2025-07-10 20:18:22,387] Trial

Лучшие гиперпараметры:  {'C': 0.05949001406067821, 'solver': 'liblinear', 'max_iter': 857, 'penalty': 'l2'}
Лучшая точность:  0.9502912621359223


- Лучшие гиперпараметры:  {'C': 0.05949001406067821, 'solver': 'liblinear', 'max_iter': 857, 'penalty': 'l2'}
- Лучшая точность:  0.9502912621359223

## Модель LGBM.

In [15]:
def objective(trial):
    # Определение гиперпараметра для SelectKBest
    k = trial.suggest_int('k', 1, 100)  # Количество признаков от 1 до числа всех признаков

    # Отбор признаков с помощью SelectKBest
    selector = SelectKBest(score_func=f_classif, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_val_selected = selector.transform(X_val)
   
    params = {
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 0.3, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 10.0, log=True),
        'random_state': 42,
        'objective': 'binary',  # Для бинарной классификации
        'verbose': -1  # Отключение вывода логов
    }

    # Создание модели LightGBM
    model = LGBMClassifier(**params)

    model.fit(X_train_selected, y_train)

    y_pred = model.predict(X_val_selected)
    f1_val = f1_score(y_val, y_pred)

    return f1_val

In [16]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)  

# Вывод лучших гиперпараметров
print("Лучшие гиперпараметры: ", study.best_params)
print("Лучшая точность: ", study.best_value)

[I 2025-07-11 14:20:25,786] A new study created in memory with name: no-name-6affb8d3-af97-43e9-8a53-500fa4b8ad74
D:\anaconda3\envs\practicum_new\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-07-11 14:20:27,580] Trial 0 finished with value: 0.9424530129819803 and parameters: {'k': 15, 'num_leaves': 93, 'learning_rate': 0.07230204885121799, 'n_estimators': 434, 'max_depth': 4, 'min_child_samples': 65, 'subsample': 0.5291354198890934, 'colsample_bytree': 0.959068869609087, 'reg_alpha': 2.3965811067074557e-05, 'reg_lambda': 4.365473702889757}. Best is trial 0 with value: 0.9424530129819803.
D:\anaconda3\envs\practicum_new\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
[I 2025-07-11 14:20:30,051] Trial 1 finished with value: 0.7777251184834123

Лучшие гиперпараметры:  {'k': 57, 'num_leaves': 148, 'learning_rate': 0.009978103296293521, 'n_estimators': 338, 'max_depth': 9, 'min_child_samples': 35, 'subsample': 0.66636025464634, 'colsample_bytree': 0.6498760983199332, 'reg_alpha': 0.00014957828627846294, 'reg_lambda': 0.024589531212131692}
Лучшая точность:  0.9478599221789883


- Лучшие гиперпараметры:  {'k': 57, 'num_leaves': 148, 'learning_rate': 0.009978103296293521, 'n_estimators': 338, 'max_depth': 9, 'min_child_samples': 35, 'subsample': 0.66636025464634, 'colsample_bytree': 0.6498760983199332, 'reg_alpha': 0.00014957828627846294, 'reg_lambda': 0.024589531212131692}
- Лучшая точность:  0.9478599221789883

## Моедль SVC.

In [20]:
def objective(trial):

    k = trial.suggest_int('k', 1, 100)  # Количество признаков от 1 до числа всех признаков

    # Отбор признаков с помощью SelectKBest
    selector = SelectKBest(score_func=f_classif, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_val_selected = selector.transform(X_val)
    
    # Определение пространства гиперпараметров
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    C = trial.suggest_float('C', 1e-3, 1e3, log=True)  # Параметр регуляризации
    gamma = trial.suggest_float('gamma', 1e-4, 1e1, log=True) if kernel in ['rbf', 'poly'] else 'scale'
    
    # Создание модели SVC
    svc = SVC(
        kernel=kernel,
        C=C,
        gamma=gamma,
        random_state=42
    )
    
    # Обучение модели
    svc.fit(X_train_selected, y_train)
    
    # Предсказание и оценка точности
    y_pred = svc.predict(X_val_selected)
    f1_val = f1_score(y_val, y_pred)

    return f1_val

In [21]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)  

# Вывод лучших гиперпараметров
print("Лучшие гиперпараметры: ", study.best_params)
print("Лучшая точность: ", study.best_value)

[I 2025-07-11 14:44:16,217] A new study created in memory with name: no-name-6d88634b-fdca-46ae-bed0-81002f89daf0
[I 2025-07-11 14:44:39,608] Trial 0 finished with value: 0.9152911300562308 and parameters: {'k': 29, 'kernel': 'poly', 'C': 0.27292658642248574, 'gamma': 0.015946968983812336}. Best is trial 0 with value: 0.9152911300562308.
[I 2025-07-11 14:48:23,409] Trial 1 finished with value: 0.0 and parameters: {'k': 88, 'kernel': 'poly', 'C': 0.019483400400456416, 'gamma': 0.001473230257510029}. Best is trial 0 with value: 0.9152911300562308.
[I 2025-07-11 14:48:40,135] Trial 2 finished with value: 0.945751506902586 and parameters: {'k': 27, 'kernel': 'rbf', 'C': 517.6134188749666, 'gamma': 0.00016151925176029224}. Best is trial 2 with value: 0.945751506902586.
[I 2025-07-11 14:48:51,723] Trial 3 finished with value: 0.9435280419173298 and parameters: {'k': 36, 'kernel': 'poly', 'C': 4.113811044455932, 'gamma': 0.03585941342895871}. Best is trial 2 with value: 0.945751506902586.
[I 

Лучшие гиперпараметры:  {'k': 64, 'kernel': 'linear', 'C': 0.02673525271482263}
Лучшая точность:  0.9477162293488824


- Лучшие гиперпараметры:  {'k': 64, 'kernel': 'linear', 'C': 0.02673525271482263}
- Лучшая точность:  0.9477162293488824

## Выбор модели.

Лучший показатель метрики `f1` выдала модель `LogisticRegression` c параметрами `'C': 0.05949001406067821, 'solver': 'liblinear', 'max_iter': 857, 'penalty': 'l2'`. Протестируем данную модель на тестовой выборке: 

In [7]:
model_test = LogisticRegression(
        C=0.05949001406067821,
        solver='liblinear',
        penalty='l2',
        max_iter=857,
        random_state=42
    )


In [10]:
model_test.fit(X_train, y_train)

y_pred = model_test.predict(X_test)
f1_test = f1_score(y_test, y_pred)

print(f'f1 модели на тестовой выборке: {f1_test}')

f1 модели на тестовой выборке: 0.9503435352904435


**Промежуточный вывод**

**1. Модель LogisticRegression**
- **Подход**:
  - Использована логистическая регрессия с оптимизацией гиперпараметров через библиотеку Optuna.
  - Настраиваемые параметры:
    - `C`: регуляризация (диапазон: 1e-5–1e2).
    - `solver`: оптимизаторы (`lbfgs`, `liblinear`, `saga`).
    - `max_iter`: максимальное количество итераций (100–1000).
    - `penalty`: тип регуляризации (`l1`, `l2`, в зависимости от `solver`).
  - Проведено 20 испытаний с метрикой F1 для максимизации качества на валидационной выборке.
- **Результаты**:
  - Лучшие гиперпараметры: `C=0.05949`, `solver='liblinear'`, `max_iter=857`, `penalty='l2'`.
  - Лучшая F1-метрика на валидационной выборке: **0.95029**.
  

**2. Модель LightGBM**
- **Подход**:
  - Использован градиентный бустинг (LGBMClassifier) с отбором признаков через `SelectKBest` (параметр `k` от 1 до 100).
  - Оптимизированы гиперпараметры с помощью Optuna (20 испытаний):
    - `num_leaves`, `learning_rate`, `n_estimators`, `max_depth`, `min_child_samples`, `subsample`, `colsample_bytree`, `reg_alpha`, `reg_lambda`.
  - Целевая метрика: F1 на валидационной выборке.
- **Результаты**:
  - Лучшие гиперпараметры: `k=57`, `num_leaves=148`, `learning_rate=0.00998`, `n_estimators=338`, `max_depth=9`, `min_child_samples=35`, `subsample=0.666`, `colsample_bytree=0.650`, `reg_alpha=0.00015`, `reg_lambda=0.0246`.
  - Лучшая F1-метрика на валидационной выборке: **0.94786**.
 
**3. Модель SVC**
- **Подход**:
  - Применен метод опорных векторов (SVC) с отбором признаков через `SelectKBest` (параметр `k` от 1 до 100).
  - Оптимизированы гиперпараметры с помощью Optuna (20 испытаний):
    - `kernel`: тип ядра (`linear`, `rbf`, `poly`).
    - `C`: регуляризация (1e-3–1e3, логарифмическая шкала).
    - `gamma`: параметр ядра для `rbf` и `poly` (1e-4–1e1, логарифмическая шкала).
  - Целевая метрика: F1 на валидационной выборке.
- **Результаты**:
  - Лучшие гиперпараметры: `k=64`, `kernel='linear'`, `C=0.02674`.
  - Лучшая F1-метрика на валидационной выборке: **0.94772**.

**4. Выбор модели и тестирование**
- **Выбор модели**:
  - На основе валидационной F1-метрики выбрана модель **LogisticRegression** с параметрами: `C=0.05949`, `solver='liblinear'`, `max_iter=857`, `penalty='l2'`, так как она показала наивысший результат (F1 = 0.95029).
- **Тестирование**:
  - Модель обучена на тренировочной выборке (`X_train`, `y_train`).
  - Оценка на тестовой выборке (`X_test`, `y_test`): F1-метрика = **0.95034**.
- **Вывод**:
  - Логистическая регрессия продемонстрировала высокую производительность и стабильность на тестовой выборке, подтверждая качество подбора гиперпараметров.
  - Модели LightGBM и SVC показали близкие результаты, но уступили по F1-метрике.

# Итоговы вывод:



В рамках проекта для интернет-магазина «Викишоп» была разработана модель машинного обучения для классификации комментариев на позитивные и негативные с целью выявления токсичных текстов для последующей модерации. Целью было достижение метрики F1 не менее 0.75. Работа выполнялась с использованием предобученной модели BERT (`unitary/toxic-bert`) и включала следующие этапы:

1. **Загрузка и изучение данных**  
   Данные из файла `toxic_comments.csv` содержали текст комментариев (столбец `text`) и целевой признак токсичности (столбец `toxic`). На этапе изучения подтверждена корректность структуры данных, отсутствие пропусков и необходимость предобработки текстов для дальнейшего анализа.

2. **Предобработка данных**  
   - Тексты токенизированы с использованием `BertTokenizer` (максимальная длина — 512 токенов, с добавлением специальных токенов и усечением).  
   - Созданы эмбеддинги текстов с помощью модели `unitary/toxic-bert`, оптимизированной для задачи определения токсичности. Процесс выполнялся на GPU для ускорения, с извлечением эмбеддингов для токена `[CLS]` в батчах (размер батча — 100).  


5. **Обучение моделей**  
   - Рассмотрены три модели: **LogisticRegression**, **LightGBM** и **SVC**, с оптимизацией гиперпараметров через библиотеку Optuna .  
   - **LogisticRegression**: Лучшая модель с гиперпараметрами `C=0.05949`, `solver='liblinear'`, `max_iter=857`, `penalty='l2'` показала F1 = **0.95029** на валидационной выборке и F1 = **0.95034** на тестовой выборке.  
   - Выбрана модель **LogisticRegression** как наиболее производительная и стабильная, удовлетворяющая целевому порогу F1 ≥ 0.75.

**Итоговые выводы и рекомендации**:  
- Разработанная модель на основе логистической регрессии с эмбеддингами BERT (`unitary/toxic-bert`) успешно решает задачу классификации токсичных комментариев, демонстрируя высокую точность (F1 = 0.95034 на тестовой выборке).  
- Использование предобученной модели BERT позволило эффективно извлечь информативные признаки из текстов, а оптимизация гиперпараметров через Optuna обеспечила выбор наилучших параметров.  
- Модель готова к внедрению в систему модерации «Викишоп» для автоматического выявления токсичных комментариев.  
